In [83]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [84]:
train_data_path = "dataset/train.csv"
test_data_path = "dataset/test.csv"

train_dataframe = pd.read_csv(train_data_path)
test_dataframe = pd.read_csv(test_data_path)

#train_dataframe.head(5)

In [85]:
label_encoder = LabelEncoder()

train_dataframe['Gender'] = label_encoder.fit_transform(train_dataframe['Gender'])
train_dataframe = pd.get_dummies(train_dataframe, columns=['Geography'])
x_train = train_dataframe.drop(['Exited','Surname','CustomerId'], axis=1)
y_train = train_dataframe['Exited']

#x_train.head(10)
#y_train.head(10)

test_dataframe['Gender'] = label_encoder.fit_transform(test_dataframe['Gender'])
test_dataframe = pd.get_dummies(test_dataframe, columns=['Geography'])
x_test = test_dataframe.drop(['Surname','CustomerId'], axis=1)
x_test.head(10)

,id,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,165034,586,0,23.0,2,0.00,2,0.0,1.0,160976.75,True,False,False
1,165035,683,0,46.0,2,0.00,1,1.0,0.0,72549.27,True,False,False
2,165036,656,0,34.0,7,0.00,2,1.0,0.0,138882.09,True,False,False
3,165037,681,1,36.0,8,0.00,1,1.0,0.0,113931.57,True,False,False
4,165038,752,1,38.0,10,121263.62,1,1.0,0.0,139431.00,False,True,False
5,165039,593,0,22.0,9,0.00,2,0.0,0.0,51907.72,True,False,False
6,165040,682,1,45.0,4,0.00,2,1.0,1.0,157878.67,False,False,True
7,165041,539,0,47.0,8,0.00,2,1.0,1.0,126784.29,False,False,True
8,165042,845,0,47.0,3,111096.91,1,1.0,0.0,94978.10,True,False,False
9,165043,645,1,30.0,5,0.00,2,0.0,1.0,149195.44,False,False,True


In [86]:
model = XGBClassifier(objective='binary:logistic')

model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [87]:
y_pred = model.predict(x_test)
y_pred_proba = model.predict_proba(x_test)[:,1]

In [88]:
output = x_test
output['Exited'] = y_pred_proba

In [89]:
columns_to_keep = ['id', 'Exited']

# Find columns to drop
columns_to_drop = [col for col in output.columns if col not in columns_to_keep]

# Drop the columns
output.drop(columns=columns_to_drop, inplace=True)

output.head(5)

,id,Exited
0,165034,0.035550
1,165035,0.763731
2,165036,0.013165
3,165037,0.196271
4,165038,0.311579


In [90]:
output.to_csv('output/submission.csv', index=False)